I want to convey two things in this notebook.
## 1. Don't have to be hesitant about using Loop.
They say "avoid loops!'.
But I think It's not bad idea to use loops for this competition.
Because:
* We have to use small batch inference using Time-series API.
* Loops have very small overhead for each batch.
* Loops are more flexible.
* Even loops are not so slow. 3 features are extracted within 10 minits for 100M train data, as you can see blow.

## 2. Future information should not be used.
Time-series API doesn't allow us to use information from the future.
So we should not use it, especially user statistics from future make things very bad.

In [1]:
import pickle
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict, deque
from tqdm.notebook import tqdm
import lightgbm as lgb

## setting
CV files are generated by [this notebook](https://www.kaggle.com/its7171/cv-strategy)

In [2]:
train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
valid_pickle = '../input/riiid-cross-validation-files/cv1_valid.pickle'
question_file = '../input/features/question3.csv'
debug = False
validaten_flg = False

## feature engineering

In [3]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Dropout, LSTM, Masking, Embedding, Concatenate, Input, Reshape,Flatten, AveragePooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.metrics import AUC
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda
#from tensorflow.keras.layers import merge
from tensorflow.keras.layers import multiply, Reshape
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm import tqdm
from tqdm import trange
from tensorflow.keras.utils import Sequence


In [4]:
FEATS = ['prior_question_elapsed_time','time_diff']

In [5]:
import pickle

loaded_pickle = open("../input/for-dkt/train_current_question.pkl", "rb")
train_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/train_other_feats.pkl", "rb")
train_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/train_past_part_answ.pkl", "rb")
train_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/train_past_tag_answ.pkl", "rb")
train_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/train_y.pkl", "rb")
train_y = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/train_prior_exp.pkl", "rb")
train_prior_exp = pickle.load(loaded_pickle)


loaded_pickle = open("../input/for-dkt/valid_current_question.pkl", "rb")
valid_current_question = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/valid_other_feats.pkl", "rb")
valid_other_feats = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/valid_past_part_answ.pkl", "rb")
valid_past_part_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/valid_past_tag_answ.pkl", "rb")
valid_past_tag_answ = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/valid_y.pkl", "rb")
valid_y = pickle.load(loaded_pickle)

loaded_pickle = open("../input/for-dkt/valid_prior_exp.pkl", "rb")
valid_prior_exp = pickle.load(loaded_pickle)


## modeling

In [6]:
other_input_dim = len(FEATS)
hidden_layer_size = 128
input_dim_order = 13523
prev_q_perform_dim = 27046
prev_t_perform_dim = 3041
prev_p_perform_dim = 15
prev_q_exp_dim = 3
epoch = 5

In [7]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [8]:
def dkt_build(hidden_layer_size, input_dim_order, prev_q_perform_dim, prev_t_perform_dim, prev_p_perform_dim, other_input_dim, prev_q_exp_dim):    
    masking_func = lambda inputs, previous_mask: previous_mask
    ## One hot encode question_id and answer/tag_id and answer
    #prev_q_a = Input(batch_shape = (None, None), dtype = 'int32', name = 'prev_q_a')
    #one_hot_prev_q_a = tf.one_hot(prev_q_a, prev_q_perform_dim, axis = -1)
    
    prev_t_a = Input(batch_shape = (None, None), dtype = 'int32', name = 'prev_t_a')
    one_hot_prev_t_a = tf.one_hot(prev_t_a, prev_t_perform_dim, axis = -1)
    
    prev_p_a = Input(batch_shape = (None, None), dtype = 'int32', name = 'prev_p_a')
    one_hot_prev_p_a = tf.one_hot(prev_p_a, prev_p_perform_dim, axis = -1)
    
    prev_q_exp = Input(batch_shape = (None, None), dtype = 'int32', name = 'prev_q_exp')
    one_hot_prev_q_exp = tf.one_hot(prev_q_exp, prev_q_exp_dim, axis = -1)
    
    other_input = Input(batch_shape = (None, None, other_input_dim), name= 'other_input')
    
    one_hot = Concatenate()([one_hot_prev_t_a, one_hot_prev_p_a, one_hot_prev_q_exp, other_input])
    
    masked_oh = (Masking(mask_value= 0, input_shape = (None, None, prev_t_perform_dim + prev_p_perform_dim + prev_q_exp + other_input_dim)))(one_hot)
    
    lstm_out = LSTM(hidden_layer_size, input_shape = (None, None, prev_t_perform_dim + prev_p_perform_dim + other_input_dim + prev_q_exp_dim),
                    dropout=0.2, recurrent_dropout =0.2, return_sequences = True)(masked_oh)
    
    
    dense_out = Dense(input_dim_order, input_shape = (None, None, hidden_layer_size), activation='sigmoid')(lstm_out)
    order = Input(batch_shape = (None, None), dtype = 'int32', name = 'order')
    #one hot encode
    one_hot_order = tf.one_hot(order, input_dim_order, axis = -1)
    #one_hot_order = K.sum(one_hot_order, axis = -2)
    
    merged = multiply([dense_out, one_hot_order])
    
    def reduce_dim(x):
        x = K.max(x, axis = 2, keepdims = True)
        return x

    def reduce_dim_shape(input_shape):
        shape = list(input_shape)
        shape[-1] = 1
        print ("reduced_shape", shape)
        return tuple(shape)
    
    reduced = Lambda(reduce_dim, output_shape = reduce_dim_shape, mask = masking_func)(merged)
    
    model = Model(inputs=[prev_t_a, prev_p_a, prev_q_exp, other_input, order], outputs=reduced)
    #model.compile( optimizer = 'adam',
    #                loss = 'binary_crossentropy',
    #                metrics=['accuracy',AUC()])

    return model

In [9]:
with tpu_strategy.scope():
    my_model = dkt_build(hidden_layer_size, input_dim_order, prev_q_perform_dim, prev_t_perform_dim, prev_p_perform_dim, other_input_dim, prev_q_exp_dim)
    my_model.compile( optimizer = 'adam',
                        loss = 'binary_crossentropy',
                        metrics=['accuracy',AUC()])

In [10]:
#my_model = dkt_build(hidden_layer_size, input_dim_order, prev_q_perform_dim, prev_t_perform_dim, prev_p_perform_dim, other_input_dim, prev_q_exp_dim)


In [11]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=1,mode='max') 

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="dkt_model.h5", save_best_only=True, save_weights_only=True, verbose=1)

my_model.fit([train_past_tag_answ, train_past_part_answ, train_prior_exp, train_other_feats, train_current_question], train_y,
                    validation_data=([valid_past_tag_answ, valid_past_part_answ, valid_prior_exp, valid_other_feats, valid_current_question], valid_y),
                    epochs = 20, verbose = 1, batch_size = 200, callbacks = [earlyStopping, cp_callback])

Epoch 1/20
3556/3556 [==============================] - ETA: 0s - accuracy: 0.7355 - loss: 0.5121 - auc: 0.7913
Epoch 00001: val_loss improved from inf to 0.46546, saving model to dkt_model.h5
3556/3556 [==============================] - 115s 32ms/step - accuracy: 0.7355 - loss: 0.5121 - auc: 0.7913 - val_accuracy: 0.7610 - val_loss: 0.4655 - val_auc: 0.8387
Epoch 2/20
3555/3556 [============================>.] - ETA: 0s - accuracy: 0.7432 - loss: 0.5015 - auc: 0.8033
Epoch 00002: val_loss improved from 0.46546 to 0.46257, saving model to dkt_model.h5
3556/3556 [==============================] - 107s 30ms/step - accuracy: 0.7432 - loss: 0.5015 - auc: 0.8033 - val_accuracy: 0.7632 - val_loss: 0.4626 - val_auc: 0.8414
Epoch 3/20
3556/3556 [==============================] - ETA: 0s - accuracy: 0.7458 - loss: 0.4978 - auc: 0.8073
Epoch 00003: val_loss improved from 0.46257 to 0.45962, saving model to dkt_model.h5
3556/3556 [==============================] - 111s 31ms/step - accuracy: 0.745

In [12]:
#my_model.save_weights("dkt_model.h5")

In [13]:
y_hat = my_model.predict([valid_past_tag_answ, valid_past_part_answ, valid_prior_exp, valid_other_feats, valid_current_question])
roc_auc_score(valid_y[:,-1,0],y_hat[:,-1,0])

0.7748090842227403